In [350]:
import pandas as pd
import numpy as np
from numpy.linalg import eig
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from kmeans import K_Means
# from svd import SVD

In [351]:
data = pd.read_excel('data/Course Recommendation System(1-162).xlsx')
data = data.drop(data.iloc[:, 0:6], axis=1) # Delete unnecessary columns
data_core = data.iloc[:, :5]

courses = list(data.columns)
core_courses = list(data_core.columns)
core_courses

['Data Structures and Algorithms',
 'Computer Architecture\n',
 'Discrete Mathematics\n',
 'Economics',
 'Programming-2']

In [352]:
data = data.fillna(0.5)
X = data.to_numpy()
X_core = data_core.to_numpy()
X_electives = X[:, 5:]

X_core contains our user vectors (ratings for each user in the core courses)

In [353]:
# K-means on user vectors

n_clusters = 5
n_init = 10
kmeans = K_Means(n_clusters, n_init)
kmeans.fit(X_core, 100)
kmeans.get_cluster_size()
cluster_assignment = kmeans.c_assignment

In [354]:
# X_comp = np.zeros((1, 25))

# for i in range(0, n_clusters):
#     temp = X_core[np.where(cluster_assignment == i)]
#     mean = np.mean(X_electives[np.where(cluster_assignment == i)], axis=0)
#     mean_repeated = np.repeat(mean.reshape(1, -1), temp.shape[0], axis=0)
#     X_comp = np.vstack((X_comp, np.concatenate((temp, mean_repeated), axis=1)))
    
# X_comp = X_comp[1:, :]

In [355]:
X_comp = np.zeros((1, 25))

for i in range(n_clusters):
    mean = np.mean(X[np.where(cluster_assignment == i)], axis=0).reshape(1, -1)
    X_comp = np.vstack((X_comp, mean))
    
X_comp = X_comp[1:, :]
X_comp.shape

(5, 25)

In [356]:
u, s, vh = np.linalg.svd(X_comp, full_matrices=False)
print(u.shape, s.shape, vh.shape)
np.allclose(X_comp, np.dot(u * s, vh))

(5, 5) (5,) (5, 25)


True

In [357]:
X_svd = np.dot(u * s, vh)

In [358]:
X_svd

array([[0.935     , 0.89833333, 0.85633333, 0.23333333, 0.86166667,
        0.84      , 0.765     , 0.54833333, 0.67      , 0.63333333,
        0.58      , 0.46      , 0.56      , 0.51166667, 0.48666667,
        0.43333333, 0.44666667, 0.39      , 0.42666667, 0.39      ,
        0.32833333, 0.42533333, 0.39766667, 0.31833333, 0.50333333],
       [0.7       , 0.145     , 0.34      , 0.8       , 0.845     ,
        0.69      , 0.535     , 0.47      , 0.43      , 0.425     ,
        0.605     , 0.5025    , 0.5       , 0.47      , 0.515     ,
        0.385     , 0.38      , 0.395     , 0.38      , 0.385     ,
        0.55      , 0.5775    , 0.54      , 0.55      , 0.63      ],
       [0.75625   , 0.325     , 0.425     , 0.03958333, 0.8625    ,
        0.83958333, 0.70416667, 0.59375   , 0.53083333, 0.59958333,
        0.68125   , 0.33541667, 0.42291667, 0.63333333, 0.5625    ,
        0.25416667, 0.1625    , 0.15416667, 0.2125    , 0.13333333,
        0.21041667, 0.30416667, 0.31666667, 0.

In [359]:
def get_predictions(n, X):
	for i in range(X.shape[0]):
		ind = np.argpartition(X[i, :], -n)[-n:]
		ind = ind[np.argsort(X_svd[i, :][ind])]
		print("Cluster - ", i + 1, "\n")
		print(X_svd[i, :][ind][::-1])
		print([courses[i] for i in ind[::-1]], "\n")
		
	
get_predictions(5, X_svd)

Cluster -  1 

[0.935      0.89833333 0.86166667 0.85633333 0.84      ]
['Data Structures and Algorithms', 'Computer Architecture\n', 'Programming-2', 'Discrete Mathematics\n', 'Machine Learning\n'] 

Cluster -  2 

[0.845 0.8   0.7   0.69  0.63 ]
['Programming-2', 'Economics', 'Data Structures and Algorithms', 'Machine Learning\n', 'The Web and the Mind '] 

Cluster -  3 

[0.8625     0.83958333 0.75625    0.70416667 0.68125   ]
['Programming-2', 'Machine Learning\n', 'Data Structures and Algorithms', 'Mathematics For Machine Learning\n', 'Software Production Engineering\n'] 

Cluster -  4 

[0.82592593 0.75555556 0.74444444 0.72592593 0.67962963]
['Computer Architecture\n', 'Economics', 'Programming-2', 'Machine Learning\n', 'Data Structures and Algorithms'] 

Cluster -  5 

[0.85725456 0.85570291 0.85157981 0.81868852 0.81839924]
['Economics', 'Programming-2', 'Data Structures and Algorithms', 'Computer Architecture\n', 'Discrete Mathematics\n'] 



### Manual SVD

In [360]:
# def QR_Decomposition(A):
#     n, m = A.shape # get the shape of A

#     Q = np.empty((n, n)) # initialize matrix Q
#     u = np.empty((n, n)) # initialize matrix u

#     u[:, 0] = A[:, 0]
#     Q[:, 0] = u[:, 0] / np.linalg.norm(u[:, 0])

#     for i in range(1, n):

#         u[:, i] = A[:, i]
#         for j in range(i):
#             u[:, i] -= (A[:, i] @ Q[:, j]) * Q[:, j] # get each u vector

#         Q[:, i] = u[:, i] / np.linalg.norm(u[:, i]) # compute each e vetor

#     R = np.zeros((n, m))
#     for i in range(n):
#         for j in range(i, m):
#             R[i, j] = A[:, j] @ Q[:, i]

#     return Q, R

# def svd_simultaneous_power_iteration(A, k, epsilon=0.00001):
#     n_orig, m_orig = A.shape
#     if k is None:
#         k=min(n_orig,m_orig)
#     A_orig=A.copy()
#     if n_orig > m_orig:
#         A = A.T @ A
#         n, m = A.shape
#     elif n_orig < m_orig:
#         A = A @ A.T
#         n, m = A.shape
#     else:
#         n,m=n_orig, m_orig
        
#     Q = np.random.rand(n, k)
#     # Q, _ = np.linalg.qr(Q)
#     Q, _ = QR_Decomposition(Q)
#     Q_prev = Q
#     #this part does the block power iteration
#     for i in range(1000):
#         Z = A @ Q
#         # Q, R = np.linalg.qr(Z)
#         Q, R = QR_Decomposition(Z)
#         err = ((Q - Q_prev) ** 2).sum()
#         Q_prev = Q
#         if err < epsilon:
#             break
            
#     singular_values=np.sqrt(np.diag(R))
#     # print(singular_values)
#     # singular_values = singular_values[:-5]
#     # print(singular_values)
#     #deal with different shape input matrices
#     if n_orig < m_orig: 
#         left_vecs=Q.T
#         # left_vecs = left_vecs[:, :-5]
#         print(left_vecs.shape)
#         #use property Values @ V = U.T@A => V=inv(Values)@U.T@A
#         right_vecs=np.linalg.inv(np.diag(singular_values))@left_vecs.T@A_orig
#     elif n_orig==m_orig:
#         left_vecs=Q.T
#         print(left_vecs.shape)
#         right_vecs=left_vecs
#         singular_values=np.square(singular_values)
#     else:
#         right_vecs=Q.T
#         # right_vecs = right_vecs[:-5, :]
#         print(right_vecs.shape)
#         #use property Values @ V = U.T@A => U=A@V@inv(Values)
#         left_vecs=A_orig@ right_vecs.T @np.linalg.inv(np.diag(singular_values))

#     print(left_vecs.shape)
#     print(singular_values.shape)
#     print(right_vecs.shape)
#     return left_vecs, singular_values, right_vecs

In [361]:
import numpy as np

class SVD:
    def __init__(self, A):
        self.u, self.s, self.vh = self.svd(A)

    def gram_schmidt(self, A):
        m, n = A.shape
        Q = np.zeros((m, n))
        R = np.zeros((n, n))

        for j in range(n):
            v = A[:, j]
            for i in range(j):
                R[i, j] = np.dot(Q[:, i], A[:, j])
                v = v - R[i, j] * Q[:, i]
            R[j, j] = np.linalg.norm(v, ord=2)
            Q[:, j] = v / R[j, j]

        return Q, R

    def eig(self, A, max_iter=1000, tol=1e-6):
        for _ in range(max_iter):
            Q, R = self.gram_schmidt(A)
            A = np.dot(R, Q)
            
            if np.linalg.norm(A - np.diag(np.diag(A)), ord=2) < tol:
                break

        return np.diag(A), Q

    def svd(self, A):
        m, n = A.shape

        if(m < n):
            A = A.T
            m, n = A.shape

        ATA = A.T.dot(A)
        eigenvals, V = eig(ATA)


        idx = eigenvals.argsort()[::-1]
        V = V[:, idx]

        singularvals = np.sqrt(np.maximum(eigenvals[idx], 0))
        S = np.zeros((m, n))
        # print(singularvals.shape)
        S[:min(m, n), :min(m, n)] = np.diag(singularvals)

        U = np.zeros((m, m))
        for i in range(min(m, n)):
            if singularvals[i] != 0:
                U[:, i] = A.dot(V[:, i]) / singularvals[i]

        return V, S.T, U.T

    def truncated_svd(self, A, k):
        U, S, Vt = self.svd(A)
		
        Uk = U[:, :k]
        Sk = S[:k, :k]
        Vtk = Vt[:k, :]

        return Uk, Sk, Vtk

    def return_matrices(self):
        return self.u, self.s, self.vh

    def return_trunc_matrices(self, A, k):
        return self.truncated_svd(A, k)


In [362]:
SVD = SVD(X_comp)
u, s, vh = SVD.return_matrices()
print(u.shape, s.shape, vh.shape)
print(np.allclose(X_comp, np.dot(u @ s, vh)))

(5, 5) (5, 25) (25, 25)
True


In [363]:
X_comp

array([[0.935     , 0.89833333, 0.85633333, 0.23333333, 0.86166667,
        0.84      , 0.765     , 0.54833333, 0.67      , 0.63333333,
        0.58      , 0.46      , 0.56      , 0.51166667, 0.48666667,
        0.43333333, 0.44666667, 0.39      , 0.42666667, 0.39      ,
        0.32833333, 0.42533333, 0.39766667, 0.31833333, 0.50333333],
       [0.7       , 0.145     , 0.34      , 0.8       , 0.845     ,
        0.69      , 0.535     , 0.47      , 0.43      , 0.425     ,
        0.605     , 0.5025    , 0.5       , 0.47      , 0.515     ,
        0.385     , 0.38      , 0.395     , 0.38      , 0.385     ,
        0.55      , 0.5775    , 0.54      , 0.55      , 0.63      ],
       [0.75625   , 0.325     , 0.425     , 0.03958333, 0.8625    ,
        0.83958333, 0.70416667, 0.59375   , 0.53083333, 0.59958333,
        0.68125   , 0.33541667, 0.42291667, 0.63333333, 0.5625    ,
        0.25416667, 0.1625    , 0.15416667, 0.2125    , 0.13333333,
        0.21041667, 0.30416667, 0.31666667, 0.

In [364]:
np.dot(u @ s, vh)

array([[0.935     , 0.89833333, 0.85633333, 0.23333333, 0.86166667,
        0.84      , 0.765     , 0.54833333, 0.67      , 0.63333333,
        0.58      , 0.46      , 0.56      , 0.51166667, 0.48666667,
        0.43333333, 0.44666667, 0.39      , 0.42666667, 0.39      ,
        0.32833333, 0.42533333, 0.39766667, 0.31833333, 0.50333333],
       [0.7       , 0.145     , 0.34      , 0.8       , 0.845     ,
        0.69      , 0.535     , 0.47      , 0.43      , 0.425     ,
        0.605     , 0.5025    , 0.5       , 0.47      , 0.515     ,
        0.385     , 0.38      , 0.395     , 0.38      , 0.385     ,
        0.55      , 0.5775    , 0.54      , 0.55      , 0.63      ],
       [0.75625   , 0.325     , 0.425     , 0.03958333, 0.8625    ,
        0.83958333, 0.70416667, 0.59375   , 0.53083333, 0.59958333,
        0.68125   , 0.33541667, 0.42291667, 0.63333333, 0.5625    ,
        0.25416667, 0.1625    , 0.15416667, 0.2125    , 0.13333333,
        0.21041667, 0.30416667, 0.31666667, 0.

In [365]:
u, s, vh = SVD.return_trunc_matrices(X_comp, 10)
print(u.shape, s.shape, vh.shape)
print(np.allclose(X_comp, np.dot(u @ s, vh)))

(5, 5) (5, 10) (10, 25)
True


In [366]:
np.dot(u @ s, vh)


array([[0.935     , 0.89833333, 0.85633333, 0.23333333, 0.86166667,
        0.84      , 0.765     , 0.54833333, 0.67      , 0.63333333,
        0.58      , 0.46      , 0.56      , 0.51166667, 0.48666667,
        0.43333333, 0.44666667, 0.39      , 0.42666667, 0.39      ,
        0.32833333, 0.42533333, 0.39766667, 0.31833333, 0.50333333],
       [0.7       , 0.145     , 0.34      , 0.8       , 0.845     ,
        0.69      , 0.535     , 0.47      , 0.43      , 0.425     ,
        0.605     , 0.5025    , 0.5       , 0.47      , 0.515     ,
        0.385     , 0.38      , 0.395     , 0.38      , 0.385     ,
        0.55      , 0.5775    , 0.54      , 0.55      , 0.63      ],
       [0.75625   , 0.325     , 0.425     , 0.03958333, 0.8625    ,
        0.83958333, 0.70416667, 0.59375   , 0.53083333, 0.59958333,
        0.68125   , 0.33541667, 0.42291667, 0.63333333, 0.5625    ,
        0.25416667, 0.1625    , 0.15416667, 0.2125    , 0.13333333,
        0.21041667, 0.30416667, 0.31666667, 0.